In [ ]:
from datasets import load_dataset
yelp_data = load_dataset("yelp_review_full")
yelp_data = yelp_data['train'][:15000] # Train (our limit)
#yelp_data = yelp_data['train'][15000:16000] # Validation small
#yelp_data = yelp_data['test'][15000:20000] # Validation medium
#yelp_data2 = yelp_data['train'][15000:]  # Validation large

In [ ]:
id_to_char = ['<PAD>', '<CLS>', '<SEP>', '<MASK>', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', \
              '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']
char_to_id = {c:i for i,c in enumerate(id_to_char)}

In [ ]:
input_ids = [] # Ids for each character
encoder_mask = [] # Mask for <PAD> tokens
word_idx = [] # Indexes of ends of each word
num_words = [] # Number of words
num_chars = [] # Number of characters
len_limit = 256
for text in yelp_data['text']:
    text = text.lower()
    d = [char_to_id['<CLS>']] # for input_ids
    w = [-1] # for word_idx, -1 for <CLS>
    n = 1 # for num_words
    for j, c in enumerate(text):
        # This part ignores the space in the beginning of new sentence, ...'s and ,,,'s.
        if len(d)==1 and c in [' ', '.', ',']:
            continue
        
        # End of sentence
        if c in ['.','?','!']:
            d.append(char_to_id[c]) # add last char
            w.append(n)
            len_d = len(d)-1 # Length except <CLS> and <SEP>
            d.append(char_to_id['<SEP>'])
            if len(d)<=len_limit: # Only add sentences with acceptable length
                mask = [0]*len(d) + [1]*(len_limit-len(d)) # Create mask
                d += [char_to_id['<PAD>']]*(len_limit-len(d)) # PAD current sentence
                w.append(-1) # <SEP>
                w += [-1]*(len_limit-len(w)) 
                
                # Append all to dataset
                input_ids.append(d)
                encoder_mask.append(mask)
                word_idx.append(w)
                num_words.append(n)
                num_chars.append(len_d)
                
            # Reset variables to start a new sentence
            d = [char_to_id['<CLS>']]
            w = [-1]
            n = 1
        else:
            d.append(char_to_id[c])
            if c==' ':
                w.append(0)
                n+=1
            else:
                w.append(n)

In [ ]:
# Save dataset
import pandas as pd
dataset = {"input_ids":input_ids, "encoder_mask":encoder_mask, "word_idx":word_idx, "num_words":num_words, "num_chars":num_chars}
dataframe = pd.DataFrame(dataset).to_pickle("-your path-")
#dataframe.to_csv("./dataset/charYelp")

In [ ]:
print("Created sentences:", len(input_ids))